# ICRA Tutorial 4, Contradictory Prior Data

**API Version: JuliaLang NavAbilitySDK.jl**

## Introduction

Many industrial robotic systems today are constrained to very rigid automation regimes.  Production/manufacturing lines are often difficult to reconfigure to different or new operations.  It can also be challenging for the robotic systems to react in real-time based on sensor data.  One of the critical requirements for expanding robotic operations beyond the rigid brute force automation is the ability to incorporate/restrain robotic operations with prior knowledge or data.  This tutorial shows how prior data can be used to support as well as constrain robotic state-estimation tasks, while also gaining robustness to contradictory data.  This tutorial will show how to combine prior and new measurement data through multi-hypothesis robustness.

## Application: Beyond "Brute-Force" Automation while Retaining Prior Knowledge

Conventional manufacturing and warehouse automation technologies operate via "brute force" or "playback" control strategies where robot operations follow a very narrow preprogrammed path.  One of the major challenges in AI development for manufacturing and warehouse applications is to make the high value robotic equipment more flexible / reconfigurable so that the systems can more rapidly be reconfigured between different tasks.  

One of the key challenges to expanding the automation software capabilities requires a balance between 
- programming the robot with human readible prior information about the task; as well as 
- letting the robotic automation handle more of the fine grain numerical operations relating to the task at hand.

The figure below shows a motivational example where prior knowledge---i.e. the corners of a rectangle and door opening---is to be used for robotic mapping operations.

![Tutorial 4 Room Prior](../../static/icra-4/room-prior.png)

The figure above illustrates some robotic equipment (a.k.a. the robot) which must be localized relative to a known rectangular shape.  For simplicity, we introduce the door orientation as the only ambiguity which must be mapped by the robot -- i.e. we know there is a door, but have not yet verified the hinging orientation.

The challenge of this tutorial is localizing and mapping the robot relative to the object as efficiently as possible, and have the solution be robust to possibly contradictory prior data.  For the sake of illustration, we simplify the possible contradiction to the door orientation only, and invite the interested reader to see the [NavAbility Construction Application example](https://www.navability.io/applications/construction/) and associated publications for a more elaborate discussion on navigation-affordances.

### Prior (Contradictory) Data

This tutorial will demonstrate through a simplified example how prior (contradictory) data can be leveraged in a localization and mapping solution using factor graphs.  We want to pre-load our robotic system with some basic information about the environment in which it will be operating so that it can more easily resolve it's position and navigate around.

#### Room with a Door

Prior info:
- A rectangular room with the 4 corners as defining known navigation affordances,
- An orientation sensitive door affordance which should* be hinged on the left side (which is buried in the data as an easter egg for the solver to discover as valid or contradictory).

## Import Packages

The first step load all the necessary packages, as listed in code blocks hereafter.  The following commented code block is only necessary if your environment does not have the necessary packages installed for whatever reason (uncomment and run once if package are not installed).

In [1]:
# # install if necessary
# import Pkg; Pkg.add("NavAbilitySDK")

In [ ]:
using NavAbilitySDK

## Build a Multi-hypothesis Factor Graph

We start with an empty factor graph object, which is created with default solver parameters.

In [ ]:
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "guest@navability.io"
robotId = "SDKjl_"*(string(uuid4())[1:4])

# also create a client connection
client = NavAbilityHttpsClient()

# You'll need a unique session number each time you run a new graph
sessionId = "Tutorial4_"*(string(uuid4())[1:4])

# context is the object to use below
context = Client(userId,robotId,sessionId)

Also a few basic noise parameters

In [ ]:
prior_distr= diagm([0.1, 0.1, 0.01].^2)
dual_distr= diagm([1, 1, sqrt(pi)].^2)

### Loading Known Prior Data

Next, add to the factor graph the four corner prior knowledge about the room, namely `c0`, `c1`, `c2`, `c3`, each with their expected location in the world via `PriorPose2` unary factor:

In [ ]:
addVariable!(fg, :c0, Pose2)
addFactor!(fg, [:c0], PriorPose2(MvNormal([0.,0, 0], prior_distr)))

addVariable!(fg, :c1, Pose2)
addFactor!(fg, [:c1], PriorPose2(MvNormal([20.,0, pi/2], prior_distr)))

addVariable!(fg, :c2, Pose2)
addFactor!(fg, [:c2], PriorPose2(MvNormal([20.,10, pi], prior_distr)))

addVariable!(fg, :c3, Pose2)
addFactor!(fg, [:c3], PriorPose2(MvNormal([0.,10, -pi/2], prior_distr)))

In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
MapVizApp(context)

In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

In [ ]:

addVariable(client, context, Variable("l0", :Pose2))
addVariable(client, context, Variable("l1", :Pose2))
addVariable(client, context, Variable("l2", :Pose2))
addVariable(client, context, Variable("l3", :Pose2))
addVariable(client, context, Variable("l4_h1", :Pose2))
addVariable(client, context, Variable("l4_h2", :Pose2))

prior_distr= diagm([0.1, 0.1, 0.01].^2)

f = Factor("l0f1", "PriorPose2", ["l0"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([0.,0, 0], prior_distr)))
addFactor(client, context, f)
f = Factor("l1f1", "PriorPose2", ["l1"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20.,0, pi/2], prior_distr)))
addFactor(client, context, f)
f = Factor("l2f1", "PriorPose2", ["l2"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20.,10, pi], prior_distr)))
addFactor(client, context, f)
f = Factor("l3f1", "PriorPose2", ["l3"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([0.,10, -pi/2], prior_distr)))
addFactor(client, context, f)
f = Factor("l4_h1f1", "PriorPose2", ["l4_h1"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20., 4, 0], prior_distr)))
addFactor(client, context, f)
f = Factor("l4_h2f1", "PriorPose2", ["l4_h2"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20., 4, pi], prior_distr)))
addFactor(client, context, f)

In [ ]:

addVariable(client, context, Variable("x0", :Pose2))

f = Factor("x0l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x0", "l0", "l1", "l2", "l3"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([-2.,-2, 0], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
addFactor(client, context, f)


In [ ]:
solveSession(client, context)

In [ ]:


addVariable(client, context, Variable("x1", :Pose2))

f = Factor("x0x1f1", 
           "Pose2Pose2", 
           ["x0", "x1"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([4.,0, pi/2], diagm([0.5, 0.5, 0.05].^2))))
addFactor(client, context, f)


f = Factor("x1l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x1", "l0", "l1", "l2", "l3"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([-2.,-4, 0], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
addFactor(client, context, f)



In [ ]:
solveSession(client, context)


In [ ]:

addVariable(client, context, Variable("x2", :Pose2))

f = Factor("x1x2f1", 
           "Pose2Pose2", 
           ["x1", "x2"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([16.,0, 0], diagm([0.5, 0.5, 0.05].^2))))
addFactor(client, context, f)


f = Factor("x2l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x2", "l0", "l1", "l2", "l3"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([2.,-4, pi/2], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
addFactor(client, context, f)





In [ ]:
solveSession(client, context)


In [ ]:


f = Factor("x2l4f1", 
           "Pose2Pose2", 
           ["x2", "l4_h1", "l4_h2"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([2.,0, pi], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.5, 0.5])
addFactor(client, context, f)


In [ ]:
solveSession(client, context)
